<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GS-for-NB" data-toc-modified-id="GS-for-NB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GS for NB</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-SVM" data-toc-modified-id="GS-for-SVM-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>GS for SVM</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-LR" data-toc-modified-id="GS-for-LR-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>GS for LR</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-RF" data-toc-modified-id="GS-for-RF-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>GS for RF</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#VISUAL" data-toc-modified-id="VISUAL-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>VISUAL</a></span></li></ul></div>

In [16]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Text-Classification/code")
!pip install pyLDAvis
!pip install gensim
!pip install pandas==1.3.0
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [17]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [18]:
nb_name = "svm_nb_lr_RF_baseline"

In [19]:
import numpy as np 
import pandas as pd
import random

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.pipeline import Pipeline

from classification_utils import *
from clustering_utils import *
from eda_utils import *
from nn_utils_keras import *
from feature_engineering_utils import *
from data_utils import *
setup_seed_ml(2021)


In [15]:
train, test = load_data()
# train, upsampling_info = upsampling_train(train)

# train_text, train_label = train_augmentation(train, select_comb=[['text'], ['reply', 'reference_one']])
train_text, train_label = train_augmentation(train, select_comb=[['text']])
test_text, test_label = test['text'], test['label']

# test_text = test_text.apply(lambda x: normal_string(x))
# train_text = train_text.apply(lambda x: normal_string(x))


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']
combination 1 train:  ['text']


In [20]:
####################################
### label mapper
####################################
labels = sorted(train_label.unique())
label_mapper = dict(zip(labels, range(len(labels))))
train_label = train_label.map(label_mapper)
test_label = test_label.map(label_mapper)
y_train = train_label.values
y_test = test_label.values

print(train_text.shape)
print(test_text.shape)
print(train_label.shape)
print(test_label.shape)

(11083,)
(7761,)
(11083,)
(7761,)


# GS for NB

In [21]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#               'clf__alpha': [0.6, 0.8, 1]}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"NB_cv_result_{nb_name}.xlsx")
# set(y_test)

## Best params

In [22]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, stop_words=True, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# tfidf_vectorizer(train_text, test_text, min_df=2, max_df=100)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative 
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = MultinomialNB(alpha=0.6)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

num of words: 37982
X_train.shape (11083, 37982)
X_test.shape (7761, 37982)
classification_report:
f1:  {'micro avg': 0.7292874629558047, 'macro avg': 0.7300994438386448} 

                          precision  recall  f1-score  support
alt.atheism                   0.830   0.702     0.761    319.0
comp.graphics                 0.735   0.712     0.723    389.0
comp.os.ms-windows.misc       0.801   0.655     0.721    394.0
comp.sys.ibm.pc.hardware      0.623   0.814     0.706    392.0
comp.sys.mac.hardware         0.861   0.808     0.834    385.0
comp.windows.x                0.842   0.823     0.832    395.0
misc.forsale                  0.811   0.838     0.825    390.0
rec.autos                     0.918   0.904     0.911    395.0
rec.motorcycles               0.940   0.945     0.942    398.0
rec.sport.baseball            0.356   0.957     0.519    397.0
rec.sport.hockey              1.000   0.046     0.088    827.0
sci.crypt                     0.842   0.927     0.882    396.0
sci.elec

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,1.322233,0.300701,0.852492,0.963909,0.819921,0.958975,0.783167,0.936968,0.819921,0.958975
1,1.151287,0.380728,0.875496,0.960201,0.847348,0.953561,0.813827,0.925500,0.847348,0.953561
2,1.125269,0.393063,0.888496,0.963257,0.865031,0.958013,0.825747,0.939860,0.865031,0.958013
3,1.294852,0.342963,0.867617,0.963611,0.848736,0.958018,0.803081,0.939564,0.848736,0.958018


# GS for SVM

In [23]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#              'clf__penalty':['l2'], 'clf__C':[1,2,3]}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"SVC_cv_result_{nb_name}.xlsx")

## Best params

In [24]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, stop_words=False, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=3, dual=True)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
cv

num of words: 38308
X_train.shape (11083, 38308)
X_test.shape (7761, 38308)
classification_report:
f1:  {'micro avg': 0.8514366705321479, 'macro avg': 0.8403710771549164} 

                          precision  recall  f1-score  support
alt.atheism                   0.841   0.793     0.816    319.0
comp.graphics                 0.747   0.797     0.771    389.0
comp.os.ms-windows.misc       0.787   0.734     0.760    394.0
comp.sys.ibm.pc.hardware      0.743   0.730     0.736    392.0
comp.sys.mac.hardware         0.814   0.873     0.842    385.0
comp.windows.x                0.876   0.803     0.838    395.0
misc.forsale                  0.852   0.903     0.877    390.0
rec.autos                     0.909   0.911     0.910    395.0
rec.motorcycles               0.957   0.957     0.957    398.0
rec.sport.baseball            0.814   0.970     0.885    397.0
rec.sport.hockey              0.992   0.886     0.936    827.0
sci.crypt                     0.931   0.922     0.926    396.0
sci.elec

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,3.325843,0.403172,0.876287,0.999773,0.874414,0.999759,0.871787,0.999772,0.874414,0.999759
1,3.367836,0.392958,0.920398,0.999547,0.915193,0.999519,0.917482,0.999547,0.915193,0.999519
2,3.336643,0.299839,0.918826,0.999774,0.915554,0.999759,0.912954,0.999775,0.915554,0.999759
3,3.404069,0.379425,0.885260,0.999661,0.884477,0.999639,0.880854,0.999661,0.884477,0.999639


In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text,stop_words=False, binary=True, min_df=1)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=3.0, dual=True)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
cv

num of words: 130097


# GS for LR

In [ ]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr', max_iter=1000))])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#              'clf__penalty':['l2'], 'clf__C':[1,2,3], 'clf__solver': ['liblinear', 'newton-cg', 'sag', 'saga' , 'lbfgs']}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"LR_1_cv_result_{nb_name}.xlsx")

In [ ]:
# metric = "f1_macro"

# text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr'))])
# parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
#              'clf__penalty':['elasticnet'], 'clf__C':[1,2,3], 'clf__l1_ratio':[0.2, 0.5,0.8],'clf__solver': ['saga']}
# gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
# gs_clf = gs_clf.fit(train_text, y_train)

# for param_name in gs_clf.best_params_:
#     print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

# print("best f1 score: {:.3f}".format(gs_clf.best_score_))
# cv_results = pd.DataFrame(gs_clf.cv_results_)
# cv_results.to_excel(f"LR_2_cv_result_{nb_name}.xlsx")

## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=1)
X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LogisticRegression(penalty="l2", multi_class='ovr', max_iter=1000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

# GS for RF

In [ ]:
from xgboost import XGBClassifier

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), 
                     ('clf', XGBClassifier(objective='multi:softmax', n_estimators=500, num_class=20, learning_rate=0.075, \
                                           colsample_bytree=0.7, subsample=0.8, eval_metric='merror'))])

parameters = {
    'tfidf__min_df': [1, 3], 'tfidf__stop_words': ['english'], 'tfidf__use_idf': [True], 'tfidf__binary': [True],
    #'svd__n_components': (250, 500, 750),
    #'clf__n_estimators': (250, 500, 750),
    'clf__max_depth': (4, 6, 8),
    'clf__min_child_weight': (1, 5, 10),
    #'clf__alpha': (0.00001, 0.000001),
    #'clf__penalty': ('l2', 'elasticnet'),
    #'clf__max_iter': (10, 50, 80),
}

gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"RF_1_cv_result_{nb_name}.xlsx")

## Best params

In [ ]:
from xgboost import XGBClassifier

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative 
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)

# param = {'max_depth': 4, 'objective': 'multi:softmax', 'clf__min_child_weight': 1, 
#          'booster':'gblinear', # linear model
#          'alpha': 0.0001,  # alpha is the L1 regularizer
#          'lambda': 1} # lambda is the L2 regularizer
# param['num_class'] = len(labels)
# param['eta'] = 0.1 # # scale weight of positive examples
# param['eval_metric'] = ['auc', 'ams@0']


clf = XGBClassifier(objective='multi:softmax', max_depth=4,min_child_weight=1, n_estimators=500, num_class=20, learning_rate=0.075, \
                                           colsample_bytree=0.7, subsample=0.8, eval_metric='merror')
# clf =  xgb.XGBClassifier(param)

In [ ]:
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
classification_report = evaluation_report(y_test, pred, labels=labels)
# cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
# pred_proba = clf.predict_proba(X_test)
# roc_auc(y_test, pred_proba)
# cv

In [ ]:
# clf.save_model("xgb.bin")
# clf.load_model("xgb.bin")
xgb.plot_importance(clf)
xgb.to_graphviz(clf, num_trees=2)

# VISUAL

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import precision_recall_fscore_support, classification_report, roc_curve, auc, precision_recall_curve
# mapping from integer feature name to original token string
feature_names = vectorizer.get_feature_names()

# Feature reduction with Kbest features based on chi2 score
ch2 = SelectKBest(chi2, k=50000)
X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)
if feature_names:
    # keep selected feature names
    feature_names = [feature_names[i] for i in ch2.get_support(indices=True)]

In [ ]:

p = (clf.predict_proba(test_dtm))[:, 1] 
p_binary = clf.predict(test_dtm)

p_label = 2
fpr, tpr, thresholds = roc_curve(test_y, p, pos_label=p_label)
auc_score = auc(fpr, tpr)

pre, rec, thresholds = precision_recall_curve(test_y, p, pos_label=p_label)

# calculate auc
prc_score = auc(rec, pre)

if show_plots:

    print(classification_report(test_y, p_binary))

    plt.subplot(1, 2, 1)

    plt.plot(fpr, tpr, color='darkorange', lw=2)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('AUC')

    plt.subplot(1, 2, 2)
    plt.plot(rec, pre, color='darkorange', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('PRC')
    plt.tight_layout()

    plt.show()

print("AUC: {:.2%}, PRC: {:.2%}".format(auc_score, prc_score))
